## Introduction

**Objective**

This script basically preprocesses/cleans the raw textual content of our news articles, in order to reduce unwanted variation at the character and word levels. Obviously the preprocessing operations are particular to the Arabic language and to our particular corpus. These operations are mainly:

- orthographic normalization of arabic characters that are commonly written in various ways, such as the alif character.
- removal of diacritic marks and tatwil character, which are useless to word meaning.
- remove urls and redundant whitespace
- remove remnant html markup text

**Pipeline**

1. read articles raw data from a pickle file (as a dataframe)
2. preprocess the text content
3. append clean data to dataframe and store in a new pickle file

In [8]:
import pandas as pd
import re

In [9]:
from arabic_nlp_utils import remove_tatweel, remove_diacritics, normalize_arabic_letters

## Read Raw Data

In [2]:
import pickle

with open('../pickle/articles_db.pickle', 'rb') as f:
    articles_data_df = pickle.load(f)

articles_data_df.shape

(7485, 6)

In [4]:
articles_data_df.head()

,title,hyperlink,date,date_dt,filename,raw_text
0,جريمة بأطوار غريبة..استيقظ الزوج ليلا فوجد زو...,https://www.assarih.com/a161665--جريمة-بأطوار-...,26 فيفري 2020,2020-02-26,www.assarih.com___جريمة_بأطوار_غريبة__استيقظ_ا...,googletag.cmd.push(function() { googletag.disp...
1,يحدث في تونس : تلميذ يعتدي على زميلته بـ«لام» ...,https://www.assarih.com/a161660-يحدث-في-تونس--...,26 فيفري 2020,2020-02-26,www.assarih.com__يحدث_في_تونس___تلميذ_يعتدي_عل...,googletag.cmd.push(function() { googletag.disp...
2,حملة أمنية في بنزرت (صور),https://www.assarih.com/a161656-حملة-أمنية-في-...,26 فيفري 2020,2020-02-26,www.assarih.com__حملة_أمنية_في_بنزرت__صور_.txt,https://www.assarih.com/a158092-حملة-أمنية-في-...
3,"تزامنا مع التخوفات من ""كورونا"" : وفاة جديدة با...",https://www.assarih.com/a161648-تزامنا-مع-التخ...,26 فيفري 2020,2020-02-26,www.assarih.com__تزامنا_مع_التخوفات_من__كورونا...,googletag.cmd.push(function() { googletag.disp...
4,تورط في 10 قضايا تحويل وجهة : سفاح النساء بالا...,https://www.assarih.com/a161647-تورط-في-10-قضا...,26 فيفري 2020,2020-02-26,www.assarih.com__تورط_في_10_قضايا_تحويل_وجهة__...,googletag.cmd.push(function() { googletag.disp...


In [5]:
articles_data_df.tail()

,title,hyperlink,date,date_dt,filename,raw_text
7480,جزائريون بالطارف منعوا 20 سيارة تونسية من العب...,https://www.assarih.com/a129489-جزائريون-بالطا...,12 مارس 2019,2019-03-12,www.assarih.com__جزائريون_بالطارف_منعوا_20_سيا...,https://www.assarih.com/a129489-جزائريون-بالطا...
7481,أميرها إمام جامع : محاكمة كتيبة إرهابية خططت ل...,https://www.assarih.com/a129488-أميرها-إمام-جا...,12 مارس 2019,2019-03-12,www.assarih.com__أميرها_إمام_جامع___محاكمة_كتي...,https://www.assarih.com/a129488-أميرها-إمام-جا...
7482,خاص/ القصرين : معركة بين دوريات أمنية ومروجي م...,https://www.assarih.com/a129487-خاص--القصرين--...,12 مارس 2019,2019-03-12,www.assarih.com__خاص__القصرين___معركة_بين_دوري...,https://www.assarih.com/a129487-خاص--القصرين--...
7483,في البحيرة : أثيوبية تضع في فم مولودها كمية من...,https://www.assarih.com/a129486-في-البحيرة---أ...,12 مارس 2019,2019-03-12,www.assarih.com__في_البحيرة___أثيوبية_تضع_في_ف...,https://www.assarih.com/a129486-في-البحيرة---أ...
7484,في المنستير: ينكّل بوالدته ويضعها على السرير ب...,https://www.assarih.com/a129485-في-المنستير--ي...,12 مارس 2019,2019-03-12,www.assarih.com__في_المنستير__ينك_ل_بوالدته_وي...,https://www.assarih.com/a129485-في-المنستير--ي...


*****

## Cleanup Text

In [10]:
def cleanup_text(text):
    if text == '':
        return ''
    # orthographic normalization of certain arabic characters
    text = remove_diacritics(text)
    text = remove_tatweel(text)
    text = normalize_arabic_letters(text)
    
    ## remove urls
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)

    ## TO DO: light stemming based on common prefixes

    # Replace special markup text between {}
    text = re.sub('[a-zA-Z]+[a-zA-Z\-\s\d{}();\.\']+', ' ', text)

    # Remove redundant white spaces
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text

In [11]:
L = ["بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيمِ" ,  " ؤ ئ إ أ ٱ آ واسى في" , 'جـــمـــيـــل' , "مواڞفات وسعر هاتف أيفون 8  ڞ الجديد"]
for x in L:
    print(x)
    print(cleanup_text(x))

بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيمِ
بسم الله الرحمٰن الرحيم
 ؤ ئ إ أ ٱ آ واسى في
ء ء ا ا ا ا واسي في
جـــمـــيـــل
جميل
مواڞفات وسعر هاتف أيفون 8  ڞ الجديد
مواڞفات وسعر هاتف ايفون 8 ڞ الجديد


In [12]:
text = "googletag.cmd.push(function() { googletag.display('div-gpt-ad-1546942376253-0'); });   "
cleanup_text(text)

''

In [14]:
articles_data_df.columns

Index(['title', 'hyperlink', 'date', 'date_dt', 'filename', 'raw_text'], dtype='object')

In [15]:
%time articles_data_df['clean_text'] = articles_data_df.raw_text.map(cleanup_text)
articles_data_df.head()

Wall time: 1.1 s


,title,hyperlink,date,date_dt,filename,raw_text,clean_text
0,جريمة بأطوار غريبة..استيقظ الزوج ليلا فوجد زو...,https://www.assarih.com/a161665--جريمة-بأطوار-...,26 فيفري 2020,2020-02-26,www.assarih.com___جريمة_بأطوار_غريبة__استيقظ_ا...,googletag.cmd.push(function() { googletag.disp...,هي جريمة اخري ولكن باطوار وتطورات غريبة تلك ال...
1,يحدث في تونس : تلميذ يعتدي على زميلته بـ«لام» ...,https://www.assarih.com/a161660-يحدث-في-تونس--...,26 فيفري 2020,2020-02-26,www.assarih.com__يحدث_في_تونس___تلميذ_يعتدي_عل...,googletag.cmd.push(function() { googletag.disp...,تمكنت دورية تابعة لمركز الحرس الوطني ببومهل من...
2,حملة أمنية في بنزرت (صور),https://www.assarih.com/a161656-حملة-أمنية-في-...,26 فيفري 2020,2020-02-26,www.assarih.com__حملة_أمنية_في_بنزرت__صور_.txt,https://www.assarih.com/a158092-حملة-أمنية-في-...,صور من حملة امنية بمدينة بنزرت واكبتها الصريح ...
3,"تزامنا مع التخوفات من ""كورونا"" : وفاة جديدة با...",https://www.assarih.com/a161648-تزامنا-مع-التخ...,26 فيفري 2020,2020-02-26,www.assarih.com__تزامنا_مع_التخوفات_من__كورونا...,googletag.cmd.push(function() { googletag.disp...,تم اليوم الاربعاء 26 فيفري 2020 تسجيل حالتي اص...
4,تورط في 10 قضايا تحويل وجهة : سفاح النساء بالا...,https://www.assarih.com/a161647-تورط-في-10-قضا...,26 فيفري 2020,2020-02-26,www.assarih.com__تورط_في_10_قضايا_تحويل_وجهة__...,googletag.cmd.push(function() { googletag.disp...,تمكن اعوان واطارات شرطة رياض النصر باريانة من ...


In [17]:
articles_data_df.raw_text[0]

'googletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1546942376253-0\'); });    هي جريمة أخرى ولكن بأطوار وتطورات غريبة تلك التي شهدتها مدينة الجيزة، وذلك بعد أن أقدم زوج بالتعاون مع والده على قتل زوجته، وتم إيقافهما على الفور من قبل وحدات الأمن ليزعم الزوج أنه قتلها بعدما شاهدها تمارس الجنس مع والده! وأوضح الزوج المتهم في أقواله أمام النيابة العامة، إنه هناك خلافات شبه مستمرة بينه وبين زوجته؛ بسبب اكتشافه سوء سلوكها.  أضاف المتهم في ذات الأقوال أنه يقيم صحبة زوجته وأولادهما وأولاده من زوجة سابقة في شقة واحدة رفقة والده البالغ من العمر 70 عامًا؛ حيث يقيم هو وزوجته والأطفال في غرفة واحدة، والغرفة الأخرى يقيم بها والده؛ وبسبب هطول الأمطار المستمرة خلال الأيام الماضية على المدينة اضطر المتهم لترك غرفته، والنوم جميعًا برفقة والده على فراش واحد، مستخدمين غطاء واحد ليقيهم من البرد. واصل المتهم اعترافاته، مشيرا إلى أنه استيقظ ليلًا على أصوات وهمهمات ليتفاجأ بزوجته ووالده اللذان كانا ينامان بجانبه وهما بصدد ممارسة الجنس؛ وعندما شعرا باستيقاظه ادعيا النوم، فقام الزوج واصطحب الأطفال خ

In [18]:
articles_data_df.clean_text[0]

'هي جريمة اخري ولكن باطوار وتطورات غريبة تلك التي شهدتها مدينة الجيزة، وذلك بعد ان اقدم زوج بالتعاون مع والده علي قتل زوجته، وتم ايقافهما علي الفور من قبل وحدات الامن ليزعم الزوج انه قتلها بعدما شاهدها تمارس الجنس مع والده! واوضح الزوج المتهم في اقواله امام النيابة العامة، انه هناك خلافات شبه مستمرة بينه وبين زوجته؛ بسبب اكتشافه سوء سلوكها. اضاف المتهم في ذات الاقوال انه يقيم صحبة زوجته واولادهما واولاده من زوجة سابقة في شقة واحدة رفقة والده البالغ من العمر 70 عاما؛ حيث يقيم هو وزوجته والاطفال في غرفة واحدة، والغرفة الاخري يقيم بها والده؛ وبسبب هطول الامطار المستمرة خلال الايام الماضية علي المدينة اضطر المتهم لترك غرفته، والنوم جميعا برفقة والده علي فراش واحد، مستخدمين غطاء واحد ليقيهم من البرد. واصل المتهم اعترافاته، مشيرا الي انه استيقظ ليلا علي اصوات وهمهمات ليتفاجا بزوجته ووالده اللذان كانا ينامان بجانبه وهما بصدد ممارسة الجنس؛ وعندما شعرا باستيقاظه ادعيا النوم، فقام الزوج واصطحب الاطفال خارج الغرفة، وعاد حاملا سكينا سدد بها طعنة في بطن زوجته، فصرخ والده وهو يحاول الاستفسار منه علي

In [19]:
articles_data_df.raw_text[100]

"https://www.assarih.com/a161240-اعترافات-جديدة-لإمام-جامع-المرسى--دنّس-المصاحف-..تحرش-بقاصر-ورسائل-غير-أخلاقية-مع-طفلة--متابعة--\ngoogletag.cmd.push(function() { googletag.display('div-gpt-ad-1546942376253-0'); });    أكد مصدر امني موثوق للصريح اون لاين بأن إمام جامع الرحمان بجبل الخاوي بالمرسى، كشف من خلال اعترافاته عن تجاوزات جديدة من بينها الكشف عن رسائل غير أخلاقية في هاتفه الجوال يتبادلها عبر تطبيقة «الميسنجر» مع طفلة تبلغ من السن 13 عاما، وباستدعاءها تبين ان الامام كان يقدم لها دروسا في العلوم الشرعية والدينية. وبينت الابحاث من خلال المكافحة بينهما انه اعتدى عليها بالفاحشة في أكثر من مرة ووجهت اليه تهمة الاعتداء بالفاحشة على قاصر سنها دون الـ15 عاما. كما وجهت اليه تهمة الإيهام بجريمة بعد تخريبه للمسجد والاعتداء على ذاته بآلة حادة وتدنيس المصاحف.        googletag.cmd.push(function() { googletag.display('div-gpt-ad-1546942685971-0'); });"

In [20]:
articles_data_df.clean_text[100]

'اكد مصدر امني موثوق للصريح اون لاين بان امام جامع الرحمان بجبل الخاوي بالمرسي، كشف من خلال اعترافاته عن تجاوزات جديدة من بينها الكشف عن رساءل غير اخلاقية في هاتفه الجوال يتبادلها عبر تطبيقة «الميسنجر» مع طفلة تبلغ من السن 13 عاما، وباستدعاءها تبين ان الامام كان يقدم لها دروسا في العلوم الشرعية والدينية. وبينت الابحاث من خلال المكافحة بينهما انه اعتدي عليها بالفاحشة في اكثر من مرة ووجهت اليه تهمة الاعتداء بالفاحشة علي قاصر سنها دون ال15 عاما. كما وجهت اليه تهمة الايهام بجريمة بعد تخريبه للمسجد والاعتداء علي ذاته بالة حادة وتدنيس المصاحف.'

****

In [22]:
articles_data_df.shape

(7485, 7)

In [21]:
corpus_df_filepath = '../pickle/articles_db_clean.pickle'

with open(corpus_df_filepath, 'wb') as f:
    pickle.dump(articles_data_df, f)